<a href="https://colab.research.google.com/github/Kingzel/pyMD/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [4]:
# conds = pd.read_json("release_conditions.json")
evids = pd.read_json("release_evidences.json")
# train_patients1 = pd.read_csv("output_1.csv")
# train_patients2 = pd.read_csv("output_2.csv")
# train_patients3 = pd.read_csv("output_3.csv")
# train_patients4 = pd.read_csv("output_4.csv")
# train_patients5 = pd.read_csv("output_5.csv")
# train_patients = pd.concat([train_patients1, train_patients2,train_patients3,train_patients4,train_patients5], ignore_index=True)


In [5]:
evids =evids.T
# conds = conds.T
# conds.drop(['cond-name-fr', 'cond-name-eng'], axis=1, inplace=True)
evids.drop(['question_fr', 'question_en','code_question'], axis=1, inplace=True)
# print(train_patients.head())
# x = train_patients.drop(['PATHOLOGY','DIFFERENTIAL_DIAGNOSIS','INITIAL_EVIDENCE'],axis=1,inplace=False)
# # x  = train_patients['SEX']
# # x =x.dropna()
# print(x.columns)
# y = train_patients['PATHOLOGY']
# print(x.head()['EVIDENCES'])
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.35)



In [6]:
multi_evid = evids.loc[evids['data_type'] == 'M']['name'].to_list()
schoice_evid = evids.loc[evids['data_type'] == 'C']['name'].to_list()
print(multi_evid,len(multi_evid))
print(schoice_evid,len(schoice_evid))

['E_55', 'E_57', 'E_54', 'E_133', 'E_152'] 5
['E_59', 'E_56', 'E_58', 'E_130', 'E_134', 'E_132', 'E_136', 'E_135', 'E_131', 'E_204'] 10


In [7]:
m_collection =[]
for i in range(len(evids)):
  if evids.iloc[i]['data_type'] == 'M':
    # print(evids.iloc[i]['name'],evids.iloc[i]['possible-values'])
    for each_val in evids.iloc[i]['possible-values']:
      col_name = evids.iloc[i]['name']+"_@_"+each_val
      m_collection.append(col_name)


print(len((m_collection)))


676


In [8]:
c_collection =[]
for i in range(len(evids)):
  if evids.iloc[i]['data_type'] == 'C':
    if all(isinstance(each_val,int) for each_val in evids.iloc[i]['possible-values']):
      c_collection.append(evids.iloc[i]['name'])
    else:
      for each_val in evids.iloc[i]['possible-values']:
        col_name = evids.iloc[i]['name']+"_@_"+str(each_val)
        c_collection.append(col_name)
print(len(c_collection))
print(c_collection)


28
['E_59', 'E_56', 'E_58', 'E_130_@_V_11', 'E_130_@_V_86', 'E_130_@_V_107', 'E_130_@_V_138', 'E_130_@_V_156', 'E_130_@_V_157', 'E_134', 'E_132', 'E_136', 'E_135_@_V_10', 'E_135_@_V_12', 'E_131_@_V_10', 'E_131_@_V_12', 'E_204_@_V_10', 'E_204_@_V_0', 'E_204_@_V_1', 'E_204_@_V_2', 'E_204_@_V_3', 'E_204_@_V_4', 'E_204_@_V_5', 'E_204_@_V_6', 'E_204_@_V_7', 'E_204_@_V_8', 'E_204_@_V_9', 'E_204_@_V_13']


In [9]:
b_collection =[]
for i in range(len(evids)):
  if evids.iloc[i]['data_type'] == 'B':
      col_name = evids.iloc[i]['name']
      b_collection.append(col_name)


print(len(b_collection))

208


In [20]:
all_collections = b_collection+m_collection+c_collection
all_collections.extend(['AGE','SEX'])
new_df = pd.DataFrame(0, index=range(len(x)), columns=all_collections)


print(new_df.shape, new_df.head(),sep="\n\n\n")
# print(new_df[new_df['E_132']==2])

(134529, 914)


   E_91  E_53  E_159  E_129  E_154  E_155  E_210  E_140  E_51  E_75  ...  \
0     0     0      0      0      0      0      0      0     0     0  ...   
1     0     0      0      0      0      0      0      0     0     0  ...   
2     0     0      0      0      0      0      0      0     0     0  ...   
3     0     0      0      0      0      0      0      0     0     0  ...   
4     0     0      0      0      0      0      0      0     0     0  ...   

   E_204_@_V_3  E_204_@_V_4  E_204_@_V_5  E_204_@_V_6  E_204_@_V_7  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   E_204_@_V_8  E_204_@_V_9  E_204_@_V_13  AGE  SEX  
0            0            0             0    0    0 

In [21]:
for i in range(len(x)):
  patient = x.iloc[i]
  age,sex,symps =patient['AGE'],patient['SEX'], patient['EVIDENCES']
  if sex == 'M':
    sex = 0
  elif sex == 'F':
    sex = 1
  new_df.loc[i, 'AGE'],new_df.loc[i, 'SEX'] = age,sex
  symps = eval(symps)
  for symp in symps:
    if symp in new_df.columns:
      new_df.loc[i, symp] = 1
    else:
      t_index = symp.index('@')-1
      intensity = int(symp[symp.index('@')+2:])
      t_column = symp[0:t_index]
      new_df.loc[i, t_column] = intensity


print(new_df.head())




   E_91  E_53  E_159  E_129  E_154  E_155  E_210  E_140  E_51  E_75  ...  \
0     0     1      0      0      0      0      0      1     0     0  ...   
1     0     1      0      0      0      0      0      0     0     0  ...   
2     0     0      0      0      0      0      0      0     0     0  ...   
3     1     1      0      0      0      0      0      0     0     0  ...   
4     1     1      0      0      0      0      0      0     0     0  ...   

   E_204_@_V_3  E_204_@_V_4  E_204_@_V_5  E_204_@_V_6  E_204_@_V_7  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   E_204_@_V_8  E_204_@_V_9  E_204_@_V_13  AGE  SEX  
0            0            0             0   49    1  
1            0

In [22]:
print(new_df.shape)

(134529, 914)


In [276]:

X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size = 0.3)
print(X_train.head()['AGE'],y_train.head(),sep="\n\n\n")

129202    47
83163     73
16952     48
45080     20
126279    68
Name: AGE, dtype: int64


129202     Acute rhinosinusitis
83163           Unstable angina
16952               Myocarditis
45080      Acute rhinosinusitis
126279    Acute pulmonary edema
Name: PATHOLOGY, dtype: object


In [1]:
clf = RandomForestClassifier(n_estimators=10, warm_start=True)
clf.fit(X_train, y_train)


NameError: ignored

In [279]:
y_pred  = clf.predict(X_test)
correct = 0
total =0
for predicted, actual in zip(y_pred,y_test):
  if predicted == actual:
    correct+=1
  total+=1
print(correct,'out of',total,correct/total,'accuracy')

44715 out of 45000 0.9936666666666667 accuracy


In [283]:
import pickle as p

with open('trained_model.bin', 'wb') as f:
    p.dump(clf, f)


In [16]:
test_patients = pd.read_csv("release_test_patients.csv")


In [17]:
x = test_patients.drop(['PATHOLOGY','DIFFERENTIAL_DIAGNOSIS','INITIAL_EVIDENCE'],axis=1,inplace=False)
y = test_patients['PATHOLOGY']


In [23]:
import pickle as p
with open('trained_model.bin', 'rb') as f:
    rf = p.load(f)

In [25]:
y_pred  = rf.predict(new_df)
correct = 0
total =0
for predicted, actual in zip(y_pred,y):
  if predicted == actual:
    correct+=1
  total+=1
print(correct,'out of',total,correct/total,'accuracy')

133584 out of 134529 0.9929754922730415 accuracy
